### 시상작업용 스크립트(비공개시상)
1. 코드
2. 시상파일을 열어서 파일,시트,추출컬럼을 지정 후 작업(n번은 작업순번 유의)
3. n번 작업결과를 join

In [1]:
import pandas as pd
import duckdb
from openpyxl import load_workbook

path='d:/사용인코드(KJ).xlsx'
cd_df = pd.read_excel(path,usecols='C,E,G',dtype={'사번':object})
cd_df.columns = ['사번','이름','지점']

In [ ]:
'''작업경로 설정 및 파일 목록 출력 '''
path ='E:/글로벌백업(180329)/04.수수료작업/202408시상/시상파일/비공개/'
path2 = '20241010_정산_손보_1차_배포(총괄)/'

import os
flist = os.listdir(f'{path}{path2}')
for i,x in enumerate(flist):
    print(i, x)
# data = pd.read_excel(f'{path}',sheet_name=None)
# data.keys()
# wb = load_workbook(r_path,read_only=True,data_only=True)
# wb.sheetnames

In [ ]:
file_nm = '20240710_AF_08_202212_메리츠화재_분급보전_배포.xlsx'
r_path = f'{path}{path2}{file_nm}'
wb = load_workbook(r_path,read_only=True,data_only=True)
print(wb.sheetnames)
# sel = input("your choice?")
# sht_name = wb.sheetnames[int(sel)-1]
# pd.DataFrame(wb[sht_name].values)

In [ ]:
'''작업대상 시트에서 추출 '''
#---------------------------------------------------------------------
no = 28
file_nm = flist[no]
print(file_nm)
r_path = f'{path}{path2}{file_nm}'
헤더, 단위 =  1, 1 #금액 단위가 천원일 경우 1000입력
ym = '202310' #시상업적월
'''작업순서를 지정하고 1번~3번까지 작업가능함..2번이상 작업한 경우 아래의 join query로 합산처리'''
sht = '2차년법인(23.7월~)' #또는 아래처럼 숫자로
seq = 1 #작업순서 ; 1.시상, 2~3: 확정 또는 환수

총괄, 시상, 사번 = 'aj','ai','o'
#환수컬럼을 지정할때만 사용, 환수시트만 집계시 시상컬럼은 0이 있는 컬럼으로 임의지정!!
환수 = '' 
환수 = None if 환수=='' else 환수
''' ------------------------------ '''
if seq == 1:
    df_2nd = pd.DataFrame()
    df_3rd = pd.DataFrame()

hdr = 헤더-1 
dic = {'총괄':총괄,'사번':사번,'시상':시상}
if 환수:
    dic['환수'] = 환수
dic = dict(sorted(dic.items(),key=lambda item: str(len(item[1]))+item[1]))
use_cols= list(dic.values())

df = pd.read_excel(f'{r_path}',header=hdr,sheet_name=sht,usecols=','.join(use_cols),
                  dtype=str)
df.columns=dic.keys()
print('read count: ', len(df))
df = df[df['총괄']=='GLAD']
df = df.astype({'시상':float})
if 환수:
    df = df.astype({'환수':float})
if "*" in str(df.iloc[0,0]):
    df['사번'] = df['사번'].str.replace('*','')

# join
c_name = 'a.환수'
query_string = f'''
     SELECT c.지점, c.이름, sum(a.시상) as 시상, c.사번 as 사번,
            sum({ 0 if 환수 is None else c_name}) as 환수
       FROM df a left outer JOIN cd_df c 
         ON a.사번 = c.사번
       GROUP BY c.지점, c.이름, c.사번 '''
df_result = duckdb.query(query_string).to_df()
df_result['시상'] = df_result['시상'] * 단위
df_result['환수'] = df_result['환수'] * 단위
print('error :', df_result[df_result['사번']==None])

if seq==1:
    df_1st = df_result.copy()
elif seq==2:
    df_2nd = df_result.copy()
elif seq==3:
    df_3rd = df_result.copy()

df_result.insert(2,"업적월",ym)
df_result['확정'] = df_result['시상']
df_result['시상'] = 0
df_result = df_result[(df_result['확정'] !=0) | (df_result['환수'] !=0)]
df_result.drop(columns='사번').to_clipboard(index=False, header=False)
print("please paste clipboard to excel")
print(df_result.head(2), '\n')
print('Total 시상:', df_result['확정'].sum() , ' 환수:', df_result['환수'].sum() if 환수 else 0)

In [ ]:
# df_2nd['시상'].sum()
# df_1st = df_2nd.copy()
# df_1st['시상'] = df_1st['시상'] * 1000
df_result['시상'] = df_result['시상'] * 1000
df_result.to_clipboard(index=False, header=False)
# dic

In [ ]:
df_1st.to_clipboard(index=False, header=False)

In [ ]:
df_result[['지점','이름','업적월','시상','환수','확정']].to_clipboard(index=False)

In [ ]:
''' 최초,확정 시상을 join '''
#확정시상이 두 개일경우 합친 후 조인
if len(df_3rd) > 0:
   df_all = pd.concat([df_2nd,df_3rd]).groupby(['지점','이름'])[['시상','환수']].sum().reset_index()
else:
   df_all = df_2nd
    
query_string = f'''
     SELECT case when a.지점 is null then b.지점 else a.지점 end as 지점,
            case when a.이름 is null then b.이름 else a.이름 end as 이름,
            a.시상 as 최초,(a.환수+ifnull(b.환수,0)) as 환수
            ,b.시상 as 확정
       FROM df_1st a  full outer JOIN df_all b
         ON a.지점 = b.지점 
         AND a.이름 = b.이름
        '''
df_res = duckdb.query(query_string).to_df()
df_res.insert(2,"업적월",ym)
# df_res.insert(4,"환수",0)
df_res.to_clipboard(index=False, header=False)
print(df_res.head(2),'\n')
print('Total 최초:', df_res['최초'].sum(), ' 환수:', df_res['환수'].sum(), '확정:', df_res['확정'].sum() )

In [ ]:
print(df_1st['시상'].sum(),' ', df_1st['환수'].sum())
print(df_all['시상'].sum(),' ', df_all['환수'].sum())
print(df_2nd['시상'].sum(),' ', df_2nd['환수'].sum())
print(df_3rd['시상'].sum(),' ', df_3rd['환수'].sum())

In [ ]:
# len(df_3rd)
df_2nd = df_2nd1.copy()
# df_result.drop(columns='사번').to_clipboard(index=False, header=False)

### 증권별  (주로 본사2차년 시상)

In [2]:
'''작업경로 설정 및 파일 목록 출력 '''
path ='E:/글로벌백업(180329)/04.수수료작업/202504시상/시상파일/비공개/'
path2 = '20250610_정산_손보_1차_배포(총괄)/'

import os
flist = os.listdir(f'{path}{path2}')
flist = [ i for i in  flist if '2차' in i ]  #2차년 목록

for i,x in enumerate(flist):
    print(i, x)

0 20250610_AF_02_202307-202402_삼성화재_영업활성화(2차년)_배포.xlsx
1 20250610_AF_04_202307-202402_MG손보_영업활성화(2차년)_배포.xlsx
2 20250610_AF_07_202307-202402_현대해상_영업활성화(2차년)_배포.xlsx
3 20250610_AF_09_202307-202402_KB손보_영업활성화(2차년)_배포.xlsx
4 20250610_AF_13_202306-202402_하나손보_영업활성화(2차년)_배포.xlsx
5 20250610_AF_14_202307-202402_롯데손보_영업활성화(2차년)_배포.xlsx
6 20250610_AF_17_202307-202401_농협손보_영업활성화(2차년)_배포.xlsx
7 20250610_AF_18_202307-202402_한화손보_영업활성화(2차년)_배포.xlsx
8 20250610_AF_20_202306-202402_메리츠화재_환수보전(2차년)_배포.xlsx
9 20250610_AF_21_202306-202401_메리츠화재_분급보전(2차년)_배포.xlsx
10 20250610_AF_19_202306-202402_메리츠화재_영업활성화(2차년)_배포.xlsx
11 20250610_AF_25_202307-202402_DB손보_영업활성화(2차년)_배포.xlsx
12 20250610_AF_27_202307-202402_흥국화재_영업활성화(2차년)_배포.xlsx
13 ~$20250610_AF_04_202307-202402_MG손보_영업활성화(2차년)_배포.xlsx


In [42]:
no = 4
file_nm = flist[no]
r_path = f'{path}{path2}{file_nm}'
헤더, 단위 =  1, 1 #금액 단위가 천원일 경우 1000입력
환수만 = True

ym = '202306' #시상업적월
sht = '23.06 2차년' #또는 아래처럼 숫자로
print(file_nm, ' ', sht)

환수, 총괄 = 'cx', 't'
증번, 사번, 실적, 시상, 상태 = 'b', 'g', 'u','v','ct'

hdr = 헤더-1 
dic = {'총괄':총괄,'사번':사번,'시상':시상,'실적':실적,'증번':증번,'환수':환수,'상태':상태}
dic = dict(sorted(dic.items(),key=lambda item: str(len(item[1]))+item[1]))
use_cols= list(dic.values())

df = pd.read_excel(f'{r_path}',header=hdr,sheet_name=sht,usecols=','.join(use_cols),
                  dtype=str)
df.columns=dic.keys()
print('read count: ', len(df))
df = df.astype({'증번':'string'})
df['환수'] = df['환수'].fillna(0)
df = df[df['총괄']=='GLAD']
if len(df)==0:
    print("no data ------------")
df = df.astype({'시상':float})
df = df.astype({'환수':float})
if "*" in str(df.iloc[0,0]):
    df['사번'] = df['사번'].str.replace('*','')
if  환수만:
    df_list = df[df['환수']!=0]
    print('Total: ', df_list['환수'].sum())
else:
    df_list = df[df['환수']!=0]
    
query_string = f'''
     SELECT c.지점, c.이름, a.증번 as 증권번호, a.실적 as 실적, a.시상 as 지급시상
            ,a.상태 as 상태
            ,(a.환수 / a.시상)* 100 as 환수율
            ,a.환수 as 환수
       FROM df_list a left outer JOIN cd_df c 
         ON a.사번 = c.사번
       ORDER BY c.지점, c.이름 '''

df_res = duckdb.query(query_string).to_df()
df_res.insert(2,"업적월",ym)
df_res.to_clipboard(index=False, header=True)
df_res.head(1)

20250610_AF_13_202306-202402_하나손보_영업활성화(2차년)_배포.xlsx   23.06 2차년
read count:  614
Total:  -29248.4


,지점,이름,업적월,증권번호,실적,지급시상,상태,환수율,환수
0,글래드가온,강은선2,202306,60424-1888,15000,30000.0,해지,-20.0,-6000.0


In [32]:
# df=df.astype({'증번':'string'})
# df.dtypes
df_res.to_clipboard(index=False, header=True)

In [27]:
df_res.groupby(['지점','이름']).sum('지급시상').reset_index().to_clipboard()

### 환수건을 복사하는 스크립트(본사시상)

In [8]:
'''작업경로 설정 및 파일 목록 출력 '''
path ='E:/글로벌백업(180329)/04.수수료작업/202501시상/시상파일/'
path2 = '20250310_본사_손보_1차_배포(총괄)/'

import os
flist = os.listdir(f'{path}{path2}')
# flist = [ i for i in  flist if '2차' in i ]

for i,x in enumerate(flist):
    print(i, x)

0 20250310_GF_01_202312-202501_MG손보_영업활성화_배포.xlsx
1 20250310_GF_02_202408-202501_AIG손보_영업활성화_배포.xlsx
2 20250310_GF_03_202312-202501_삼성화재_영업활성화_배포.xlsx
3 20250310_GF_04_202312-202501_현대해상_영업활성화_배포.xlsx
4 20250310_GF_05_202406-202501_라이나손보_영업활성화_배포.xlsx
5 20250310_GF_06_202312-202501_KB손보_영업활성화_배포.xlsx
6 20250310_GF_07_202312-202501_롯데손보_영업활성화_배포.xlsx
7 20250310_GF_08_202312-202501_메리츠화재_영업활성화_배포.xlsx
8 20250310_GF_09_202311-202501_농협손보_영업활성화_배포.xlsx
9 20250310_GF_10_202312-202501_하나손보_영업활성화_배포.xlsx
10 20250310_GF_11_202312-202501_한화손보_영업활성화_배포.xlsx
11 20250310_GF_12_202312-202501_DB손보_영업활성화_배포.xlsx
12 20250310_GF_13_202312-202501_흥국화재_영업활성화_배포.xlsx
13 ~$20250310_GF_03_202312-202501_삼성화재_영업활성화_배포.xlsx


In [11]:
'''작업경로 설정 및 파일 목록 출력 '''
def a_num(col):
    num1 = ord(col[0].upper())-65
    if len(col)==2:
        num2 = ord(col[1].upper())-65 + 26
    else: num2 = 0
    if num1 > 0 and len(col)==2:
        num1 = num1 * 26
    return num1 + num2
    
query_string = f'''
     SELECT c.지점, c.이름, a.*
       FROM df a left outer JOIN cd_df c 
         ON a.사번 = c.사번
        '''

no = 7
file_nm = flist[no]

r_path = f'{path}{path2}{file_nm}'
헤더 = 3
sht = '2411환수' #또는 아래처럼 숫자로
총괄, 사번, 환수 = 'ec','c','el'
총괄, 사번, 환수 = a_num(총괄), a_num(사번), a_num(환수)

df = pd.read_excel(f'{r_path}',skiprows=헤더-1,sheet_name=sht,dtype={사번:'str'})
cols = list(df.columns)
cols[총괄],cols[사번],cols[환수] = '총괄','사번','환수금'
df.columns = cols
df = df[(df['총괄']=='GLAD')]
df['환수금'] = df['환수금'].fillna(0)
df = df[df['환수금']!=0]

df_result = duckdb.query(query_string).to_df()
df_result.to_clipboard(index=False)
print(df_result['환수금'].sum())

0.0


In [ ]:
pd.read_excel?

In [ ]:
ym='202408'
import tabula
root_dir = 'E:/글로벌백업(180329)'
in_file = fr"{root_dir}\04.수수료작업\{ym}시상\2024년10월총괄명세서(GLAD_10일자_순번1).pdf"
df = tabula.read_pdf(in_file, pages='all')

In [ ]:
df_amt = df[9]
df_amt['금액']= df_amt['금액'].str.replace(',','')
df_amt = df_amt.astype({'금액':'float'})
print('정산시상 총액 = ',df_amt[df_amt['시상구분']=='정산']['금액'].sum())
print(' ----- details ----------')
# print(df_amt[df_amt['시상구분']=='정산'])

print('본사시상 총액 = ',df_amt[df_amt['시상구분']=='본사']['금액'].sum())

### 생보 총괄,지사 시상 지급율을 추출(기지급)

In [24]:
''' 시상지급율을 출력 '''
import pandas as pd

ym = '202412'
pols = ['00302172469']

path =f'E:/글로벌백업(180329)/04.수수료작업/{ym}시상'
path2 = f'{ym} KJ시상정산.xlsx'
in_file = path + '/' + path2
cols= ['담당','계약일', '증권번호','보험회사','월납보험료', 'GLAD', '시상율','총괄율', '지사율','총괄값', '지사값']
df = pd.read_excel(in_file, sheet_name='생보',dtype={'증권번호':str})
df[df['증권번호'].isin(pols)][cols]


,담당,계약일,증권번호,보험회사,월납보험료,GLAD,시상율,총괄율,지사율,총괄값,지사값
86,전용순,2024-12-26,00302172469,신한생명,87822.0,175644.0,2.0,0.0,0.0,0.0,0.0
